In [1]:
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold
import dateutil.easter as easter

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [3]:
from torch.utils.data import Dataset, DataLoader

In [4]:
from accelerate import Accelerator
import torch.optim as optim

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
import time

In [7]:
from tqdm.notebook import tqdm

In [8]:
import gc

## Global Variables ###

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
device

device(type='cuda', index=0)

In [11]:
from colorama import Fore, Back, Style
r_ = Fore.RED
b_ = Fore.BLUE
c_ = Fore.CYAN
g_ = Fore.GREEN
y_ = Fore.YELLOW
m_ = Fore.MAGENTA
sr_ = Style.RESET_ALL

In [12]:
BASE_DIR = '/sharedHDD/rohit/timeseries_learning/ubiquant/'
DATA_DIR = BASE_DIR+'data/parquet/'
INPUT_DIR = BASE_DIR+'input/'
WEIGHTS_DIR = BASE_DIR + 'weights/'

In [13]:
input_chunk_length = 30
output_chunk_length = 1
embedding_dim = 50 ## Investment id embedding dim

In [14]:
features = [f'f_{i}' for i in range(300)]

## Load data ##

In [15]:
%%time
train_df = pd.read_parquet(DATA_DIR+'train_low_mem.parquet')

CPU times: user 8.5 s, sys: 14.6 s, total: 23.1 s
Wall time: 4.14 s


In [16]:
investment_ids = train_df.investment_id.unique().tolist()

In [17]:
max(investment_ids)

3773

In [18]:
with open(BASE_DIR+'input/folds.pickle', 'rb') as f:
    folds = pickle.load(f)

#### Utility functions ######

In [19]:
all_features_columns = ['investment_id', 'target'] + features

In [20]:
def get_feature_array_train(rows):
    target_val = rows.target.values
    
    inp_vec = rows[all_features_columns].values
    
    return inp_vec,target_val

In [21]:
def get_feature_array_train(rows):
    target_val = rows.target.values
    
    inp_vec = rows[all_features_columns].values
    
    return inp_vec,target_val

In [22]:
def sliding_windows(X,Y,seq_length,prediction_length):
    x = []
    y = []
    for i in range(len(X)-(prediction_length+seq_length)):
        _x = X[i:(i+seq_length),:].reshape(seq_length,-1)
#         _y = Y[(i+seq_length):(i+prediction_length+seq_length),:]
        _y = Y[i+seq_length]
        x.append(_x)
        y.append(_y)

    return x,y

In [23]:
def make_sequences(df,to_exclude_ids):
    x = []
    y = []
    investment_ids = df.investment_id.unique()
    for iid in investment_ids:
        if iid not in to_exclude_ids:
            df_tmp = df[df.investment_id == iid].sort_values('time_id')
            tmp_X,tmp_Y = get_feature_array_train(df_tmp)
            _x,_y = sliding_windows(tmp_X,tmp_Y,input_chunk_length,output_chunk_length)
            x.extend(_x)
            y.extend(_y)
            
#             gc.collect()
        # else:
        #     print('Not including iid:', iid)
    
    return np.float32(np.array(x)),np.float32(np.array(y))

In [24]:
gc.collect()

180

## Modeling ##

In [25]:
config = {
    'seq_length' : input_chunk_length,
    'num_epochs' : 100,
    'lr' : 0.00001,
    'input_size' : 351,
    'hidden_size' : 351,
    'num_layers' : 1,
    'num_classes' :1, ## This is  output dimension
    'train_shuffle': True,
    'val_shuffle': True,
    'batch_size' : 4096*2,
    'best_model_name' : 'lstm',
    'bidirectional' : False,
    'only_last_hidden': True
}

In [26]:
class TSDataset(Dataset):
    
    def __init__(self,x,y):
        """
        Args:
        """
        self.x=x
        self.y=y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        
        sample = [torch.Tensor(self.x[idx]), torch.Tensor(self.y[idx])]
        return sample

#### Model ####

In [27]:
num_epochs = config['num_epochs']
lr = config['lr']
input_size = config['input_size']
hidden_size = config['hidden_size']
num_layers = config['num_layers']
num_classes = config['num_classes']
seq_length = config['seq_length']
bidirectional = config['bidirectional']
only_last_hidden = config['only_last_hidden']

#### Loss function ######

In [28]:
def pearson_loss(x, y):
    xd = x - x.mean()
    yd = y - y.mean()
    nom = (xd * yd).sum()
    denom = ((xd ** 2).sum() * (yd ** 2).sum()).sqrt()
    return 1 - nom / denom

In [29]:
class LstmTsModel(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers,seq_length):
        super(LstmTsModel, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        cat_input_dim: int = 3774
        
        self.embedding = nn.Embedding(cat_input_dim, embedding_dim)
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True,bidirectional=bidirectional)
        
        if bidirectional:
            m=2
        else:
            m=1
        
        if only_last_hidden:
            input_fc_dim = hidden_size*m
        else:
            input_fc_dim = self.seq_length*hidden_size*m
        
        print(input_fc_dim)
        
        self.fc = nn.Sequential(nn.Linear(input_fc_dim, input_fc_dim//2),
                                nn.BatchNorm1d(num_features=input_fc_dim//2),
                                nn.Dropout(0.2),
                                nn.ReLU(),
                                
#                                 nn.Linear(input_fc_dim//4, input_fc_dim//16),
#                                 nn.BatchNorm1d(num_features=input_fc_dim//16),
#                                 nn.Dropout(0.2),
#                                 nn.ReLU(),
                                
#                                 nn.Linear(input_fc_dim//16, input_fc_dim//32),
#                                 nn.BatchNorm1d(num_features=input_fc_dim//32),
#                                 # nn.Dropout(0.2),
#                                 nn.ReLU(),
                                
                                
                                nn.Linear(input_fc_dim//2, self.num_classes)
                                )

    def forward(self, x):
        # Propagate input through LSTM
        
        investment_ids = x[:,:,0].type(torch.LongTensor).to(device)
        
        embeddings = self.embedding(investment_ids)
        
        x = torch.cat([embeddings,x[:,:,1:]], dim=2)
        
        h_out, (h_n, _) = self.lstm(x)
        if only_last_hidden:
            h_out = h_out[:,-1:,:]
        
#         print(h_out.shape)
        h_out = h_out.flatten(start_dim=1)
#         print(h_out.shape)
        
        out = self.fc(h_out)
        
        return out

In [30]:
def run(model,train_dl,val_dl,fold):
    def evaluate(model,valid_loader):
        model.eval()
        valid_loss = 0
        rec_loss = 0
        with torch.no_grad():
            for i, inputs in enumerate(tqdm(valid_loader)):
                dataX = inputs[0]
                dataY = inputs[1]
                outputs = model(dataX)
                loss = criterion(outputs, dataY)
                valid_loss += loss.item()

        valid_loss /= len(valid_loader)
        return valid_loss
    
    def train_and_evaluate_loop(train_loader,model,optimizer,criterion,epoch,lr_scheduler=None,valid_loader=None, best_loss=99999):
        train_loss = 0
        for i, inputs in enumerate(tqdm(train_loader)):
            optimizer.zero_grad()
            model.train()
            
            dataX = inputs[0]
            dataY = inputs[1]
            outputs = model(dataX)
            loss = criterion(outputs, dataY)
            
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
            if lr_scheduler:
                lr_scheduler.step()
        
        train_loss /= len(train_loader)
        if valid_loader:
            valid_loss = evaluate(model,valid_loader) 
            print(f"Epoch:{epoch} |Train Loss:{train_loss}|Valid Loss:{valid_loss}")
            if valid_loss <= best_loss:
                print(f"{g_}Loss Decreased from {best_loss} to {valid_loss}{sr_}")

                best_loss = valid_loss
                torch.save(model.state_dict(), WEIGHTS_DIR+str(fold)+'_'+config['best_model_name'])
        else:
            print(f"Epoch:{epoch} |Train Loss:{train_loss}")
            
                    
        return best_loss
    
    accelerator = Accelerator()
    print(f"{accelerator.device} is used")

    
    
    optimizer = optim.Adam(model.parameters(),lr=config['lr'],amsgrad=False)
    criterion = pearson_loss
    
    # lr_scheduler = CosineAnnealingWarmupRestarts(optimizer, **config_lr)
    # lr_scheduler =  torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, **config_lr)
    lr_scheduler = None

    model,train_dl,val_dl,optimizer,lr_scheduler,criterion = accelerator.prepare(model,train_dl,val_dl,optimizer,lr_scheduler,criterion)

    best_loss = 9999999
    start_time = time.time()
    for epoch in tqdm(range(config["num_epochs"])):
        print(f"Epoch Started:{epoch}")
        best_loss = train_and_evaluate_loop(train_dl,model,optimizer,criterion,epoch,lr_scheduler,valid_loader=val_dl, best_loss=best_loss)
        
        end_time = time.time()
        print(f"{m_}Time taken by epoch {epoch} is {end_time-start_time:.2f}s{sr_}")
        start_time = end_time
        
    return best_loss, model

In [31]:
gc.collect()

20

In [ ]:
# for fold in folds.keys():
for fold in [3,4]:
    print(f'Starting for fold: {fold}{r_}')
    print(f'Preparing training data for fold: {fold}{m_}')
    train_indxs = folds[fold]['train']
    test_indxs = folds[fold]['test']
    test_f_df = train_df[train_df.index.isin(test_indxs)].reset_index(drop=True)
    train_f_df = train_df[train_df.index.isin(train_indxs)].reset_index(drop=True)
    
    print(train_f_df.shape)
    count_df = train_f_df.groupby('investment_id',as_index=False).agg(c = ('time_id', len))
    to_exclude_ids = count_df[count_df.c<input_chunk_length+1].investment_id.unique().tolist()
    
    X_train, Y_train = make_sequences(train_f_df,to_exclude_ids)
    Y_train = Y_train.reshape(-1,1)
    print(X_train.shape,Y_train.shape)
    
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)
    print(X_train.shape,Y_train.shape,X_val.shape,Y_val.shape)
    
    model = LstmTsModel(num_classes, input_size, hidden_size, num_layers,seq_length)
    train_dl = DataLoader(TSDataset(X_train, Y_train), batch_size=config['batch_size'], shuffle=config['train_shuffle'], num_workers=0)
    val_dl = DataLoader(TSDataset(X_val, Y_val), batch_size=config['batch_size'], shuffle=config['train_shuffle'], num_workers=0)
    
    best_loss, model = run(model,train_dl,val_dl,fold)
    
    gc.collect()
    

Starting for fold: 3
Preparing training data for fold: 3
(2097273, 304)
(1986639, 30, 302) (1986639, 1)
(1589311, 30, 302) (1589311, 1) (397328, 30, 302) (397328, 1)
351
cuda is used


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch Started:0


  0%|          | 0/195 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

...

Epoch:86 |Train Loss:0.7288260727512593|Valid Loss:0.8621946996258151
Time taken by epoch 86 is 159.60s
Epoch Started:87


  0%|          | 0/245 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

Epoch:87 |Train Loss:0.7268792619510572|Valid Loss:0.861464403329357
Time taken by epoch 87 is 163.86s
Epoch Started:88


  0%|          | 0/245 [00:00<?, ?it/s]